In [1]:
import gov_inertia as gi
import numpy as np

In [2]:
parameters = gi.initial_conditions

In [3]:
parameters

{'N': 1000,
 'm': 0.01,
 'k': 50,
 'G': 1,
 'H_0': 3,
 'H': 0,
 'rho': 2.3,
 'theta': 1.3,
 'F': 0.2,
 'alpha': 0.1,
 'beta': 0.1,
 'gamma': 0.1,
 'delta': 5,
 'mu_0': 0,
 'sigma': 1,
 'tau': 0.1,
 'lambda': 0.4,
 'psi': 0.3,
 'epsilon': 1,
 'zeta': 0.0001,
 'eta': 0.7}

Let's walk through the simulation and define functions to do it.

First, how many interactions does our population have?

In [4]:
def calculate_interactions(parameters):
    return parameters['m'] * parameters['k'] * parameters['N']
I = calculate_interactions(parameters)
print(I)

500.0


Next, how many interactions must be enforced?

In [5]:
def calculate_enforced_interactions(parameters, I):
    return I * parameters['F']
I_E = calculate_enforced_interactions(parameters, I)
print(I_E)

100.0


Next, what is our processing capacity?

In [6]:
def calculate_information_capacity(parameters):
    eta = parameters['eta']
    G = parameters['G']
    H = parameters['H_0'] + (parameters['rho'] * G**parameters['theta'])
    parameters['H'] = H
    return eta * G * H
B = calculate_information_capacity(parameters)
print(B)

3.7099999999999995


Okay, that's not gonna cut it. We need to scale up our government until we can enforce our laws

In [7]:
while B < I_E:
    parameters['G'] += 1
    B = calculate_information_capacity(parameters)
    print(parameters['G'], parameters['H'])

2 8.663264301386615
3 12.593685275179775
4 16.944592411895663
5 21.63755086196677
6 26.62242405985393


Now, what is our bureaucratic latency?

In [8]:
def calculate_government_latency(parameters):
    alpha = parameters['alpha']
    N = parameters['N']
    beta = parameters['beta']
    H = parameters['H']
    gamma = parameters['gamma']
    return alpha * (N**beta) * (H**gamma)
L = calculate_government_latency(parameters)
print(L)

0.2770286972297583


How much does this will exertion cost our current government?

In [9]:
def calculate_enforcement_cost(parameters):
    eps = parameters['epsilon']
    k = parameters['k']
    m = parameters['m']
    delta = parameters['delta']
    N = parameters['N']
    zeta = parameters['zeta']
    
    return eps * ((k * m)**delta) * (N**zeta)
E_C = calculate_enforcement_cost(parameters)

# that was cost of enforcement per enforceable interaction
# cost of enforcement C_E is different
C_E = E_C * I_E

# now consider latency costs as well

C_total = C_E + L

print(E_C)
print(C_E)
print(C_total)

0.031271594192758094
3.1271594192758094
3.404188116505568


Now let's see how wealthy this population is

In [17]:
def draw_population_wealth(parameters):
    mu = parameters['mu_0']
    sigma = parameters['sigma']
    N = parameters['N']
    wealth = np.random.lognormal(mean = mu, sigma = sigma, size = N)
    return wealth

wealth = draw_population_wealth(parameters)
print(sum(wealth))

1523.455047513946


There are the wealth values. In order for the government to pay to enforce interactions, it must levy a tax.

In [18]:
def levy_tax(parameters, wealth):
    revenue = wealth * parameters['tau']
    return sum(revenue)

R = levy_tax(parameters, wealth)
print(R)

152.3455047513946


In [19]:
surplus = R - C_total
print(surplus)

148.94131663488903


What do we do with our surplus?

It can be invested in infrastructure, increasing $\mu$ for potential wealth.

Invest in government, increasing $G$ and $H$. Perhaps introduce $T$ technology to improve efficiency and sensitivity scores that haven't been introduced yet.

Increase authority level by adjusting $F$ or decrease enforcement cost $\epsilon$

Accrue reserve funds for future iterations

Corruption and waste

In [20]:
mu = parameters['mu_0']
def redistribute_wealth(parameters, mu, B, L):
    mu = mu + parameters['lambda'] * np.log(1 + B) - (parameters['psi'] * L)
    return mu
mu = redistribute_wealth(parameters, mu, B, L)

In [21]:
mu

1.8071882107121549

In [15]:
def reduce_enforcement_cost(parameters, surplus):
    parameters['epsilon'] = max(1e-5, parameters['epsilon'] * np.exp(-0.001 * surplus))
    parameters['delta'] = max(1.0, parameters['delta'] - 0.005 * np.log1p(surplus))
    parameters['zeta'] = max(0.1, parameters['zeta'] - 0.005 * np.log1p(surplus))
    return parameters


Now let's grow our government $G$ to enable higher capacity of enforcement for a growing population

Recall the capacity to process interactions is 

$$ B = \eta G H$$

Now assume a per-unit cost to increase capacity $C_B$

$$C_\text{growth} = C_B \Delta{B} = C_B \eta \Delta{\left(G H\right)}$$

$H$ is a function of $G$, so we can find $\Delta B$ as 

$$
\begin{align*}
\Delta{B} = \Delta{GH} = \frac{d}{DG}\left[G \left(H_0 + \rho G^{\theta}\right)\right] &= \left(H_0 + \rho G^{\theta}\right) + G \cdot \rho \cdot \theta G^{\theta - 1} \\
              &= H_0 + \rho G^{\theta} + \rho \theta G^{\theta}
\end{align*}
$$

Then the cost to increase government becomes

$$C_\text{growth}\left(G\right) = C_B \left(H_0 + \rho G^{\theta} + \rho \theta G^{\theta}\right)$$

where $H_0$ represents the current state of hierarchical bureaucratic depth, and $G$ represents the number of governing agents to add.

In [16]:
def calculate_H(parameters):
    H_0 = parameters['H_0']
    rho = parameters['rho']
    G = parameters['G']
    theta = parameters['theta']
    H = H_0 + rho * (G**theta)
    parameters['H'] = H
    return H

def calculate_expansion_cost(parameters):
    kappa_g = parameters['kappa_g']
    kappa_h = parameters['kappa_h']
    
    G = parameters['G']
    H = parameters['H']
    
    phi = parameters['phi']
    
    cost = (kappa_g * (G**phi)) + (kappa_h * (H*phi))
    

def expand_government(parameters, surplus):
    
    eta = parameters['eta']
    
    G = parameters['G']
    H = parameters['H']
    
    current_capacity = eta * G * H
    # cost 